## Задание 1

**Написать теггер на данных с русским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы


## загрузка данных

In [1]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pyconll

import warnings
warnings.filterwarnings("ignore")

import nltk
from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [4]:
MY_DRIVE_PATH = '/content/drive/MyDrive/datasets'

TRAIN_PATH = MY_DRIVE_PATH + '/ru_syntagrus-ud-train-a.conllu'
TEST_PATH = MY_DRIVE_PATH + '/ru_syntagrus-ud-test.conllu'


In [5]:
!wget -O $TRAIN_PATH https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O $TEST_PATH https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu


--2022-11-08 15:39:32--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736565 (39M) [text/plain]
Saving to: ‘/content/drive/MyDrive/datasets/ru_syntagrus-ud-train-a.conllu’

/content/drive/MyDr 100%[===================>]  38.85M  55.9MB/s    in 0.7s    

2022-11-08 15:39:33 (55.9 MB/s) - ‘/content/drive/MyDrive/datasets/ru_syntagrus-ud-train-a.conllu’ saved [40736565/40736565]

--2022-11-08 15:39:34--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connectin

In [6]:
full_train = pyconll.load_from_file(TRAIN_PATH.replace('\\', ''))
full_test = pyconll.load_from_file(TEST_PATH.replace('\\', ''))

In [7]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [8]:
fdata_train = [[(token.form, token.upos) for token in sent] for sent in full_test[:]]

fdata_test = [[(token.form, token.upos) for token in sent] for sent in full_test[:]]

fdata_sent_test = [[token.form for token in sent] for sent in full_test[:]]
    
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)

print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [9]:
train_texts = [' '.join(token.form for token in sent) for sent in full_train]
test_texts = [' '.join(str(token.form for token in sent)) for sent in full_test]

train_labels = [' '.join(token.form for token in sent) for sent in full_train]
test_labels = [' '.join(str(token.form for token in sent)) for sent in full_test]

print('\n'.join(train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

### Default Tagger

In [10]:
default_tagger = nltk.DefaultTagger('NOUN')

default_tagger.evaluate(fdata_test)

0.23145915221945831

### Unigram Tagger

In [11]:
unigram_tagger = UnigramTagger(fdata_train)
unigram_tagger.evaluate(fdata_test)

0.972295539562881

### Bigram Tagger

In [12]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_tagger.evaluate(fdata_test)

0.9808277791491813

### Trigram Tagger

In [13]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_tagger.evaluate(fdata_test)

0.9854356948901506

In [14]:
trigram_tagger.tag(fdata_sent_test[2])

[('С', 'ADP'),
 ('одной', 'NUM'),
 ('стороны', 'NOUN'),
 (',', 'PUNCT'),
 ('квалифицированные', 'VERB'),
 ('кадры', 'NOUN'),
 ('и', 'CCONJ'),
 ('развитая', 'VERB'),
 ('производственная', 'ADJ'),
 ('инфраструктура', 'NOUN'),
 ('резко', 'ADV'),
 ('отличали', 'VERB'),
 ('Армению', 'PROPN'),
 ('от', 'ADP'),
 ('других', 'ADJ'),
 ('регионов', 'NOUN'),
 ('СССР', 'PROPN'),
 (',', 'PUNCT'),
 ('где', 'ADV'),
 ('доминировали', 'VERB'),
 ('добывающие', 'VERB'),
 ('отрасли', 'NOUN'),
 (',', 'PUNCT'),
 ('а', 'CCONJ'),
 ('экономика', 'NOUN'),
 ('строилась', 'VERB'),
 ('на', 'ADP'),
 ('поставке', 'NOUN'),
 ('сырьевых', 'ADJ'),
 ('ресурсов', 'NOUN'),
 ('.', 'PUNCT')]

### Комбинация тэггеров

In [15]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NOUN') 
tags = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tags.evaluate(fdata_test)

0.9836444309413946

### 2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов

In [16]:
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
all_test_texts = [' '.join(str(token.form for token in sent)) for sent in full_test]

all_train_labels = [' '.join(token.form for token in sent) for sent in full_train]
all_test_labels = [' '.join(str(token.form for token in sent)) for sent in full_test]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

In [17]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [18]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(str(tok[0]))
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [19]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [20]:
test_enc_labels = le.transform(test_label)

In [21]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [22]:
hvectorizer = HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=100)

In [23]:
X_train = hvectorizer.fit_transform(train_tok)

In [24]:
X_test = hvectorizer.transform(test_tok)

In [25]:
X_train.shape

(426182, 100)

In [26]:
lr = LogisticRegression(random_state=0)
lr.fit(X_train, train_enc_labels)

LogisticRegression(random_state=0)

In [27]:
pred = lr.predict(X_test)

In [28]:
accuracy_score(test_enc_labels, pred)

0.6776927507611289

In [29]:
classifier = LogisticRegression()

In [30]:
def get_model_accuracy(model):
    X_train = model.fit_transform(train_tok)
    X_test = model.transform(test_tok)
    
    classifier.fit(X_train, train_enc_labels)
    pred = classifier.predict(X_test)
    
    return accuracy_score(test_enc_labels, pred) 

In [31]:
count_vectorizer_char = CountVectorizer(ngram_range=(1,5), analyzer='char', max_features=100)
accuracy_score_ = get_model_accuracy(count_vectorizer_char)
print(f'count_vectorizer_char {accuracy_score_}')

count_vectorizer_char 0.7731804112944572


In [32]:
model = CountVectorizer(ngram_range=(1,5), analyzer='word', max_features=100)
accuracy_score_ = get_model_accuracy(model)
print(f'count_vectorizer_word {accuracy_score_}')

count_vectorizer_word 0.3774946357024856


In [33]:
model = TfidfVectorizer(ngram_range=(1,5),analyzer='char', max_features=100)
accuracy_score_ = get_model_accuracy(model)
print(f'TfidfVectorizer_char {accuracy_score_}')

TfidfVectorizer_char 0.77073087366842


In [34]:
model = TfidfVectorizer(ngram_range=(1,5),analyzer='word', max_features=100)
accuracy_score_ = get_model_accuracy(model)
print(f'TfidfVectorizer_word {accuracy_score_}')

TfidfVectorizer_word 0.37748830614789636


### 3.сравнить все реализованные методы, сделать выводы

Из одиночных теггеров лучший результат показал триграммный (0.9854) теггеры.

Комбинация теггеров чуть хуже триграммного (0.9836)

У векторайзеров дела хуже, чем у теггеров. 